In [58]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [59]:
from PIL import Image
import numpy as np
import io
import os 

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset, ConcatDataset
from torchvision.transforms import transforms
import numpy as np
from torchvision import models, datasets
from torchsummary import summary

In [61]:
from sklearn.model_selection import train_test_split
import torch

In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

### Load Data

In [63]:
class LocalData:
    def __init__ (self, clinic_id, data_range): #data_range will be removed for final code
        self.clinic_id = clinic_id
        self.path = f'../120_dataset/{clinic_id}/'
        self.range = data_range #remove for final code
        
        
    def dataset (self):
        transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize imag
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        #let's try Crop 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
            
        ])

# Load the dataset from the train folder
        train_dataset = datasets.ImageFolder(root=f'{self.path}', transform=transform)
        subset_indices = list(self.range)    #remove for final code
        train_dataset = Subset (train_dataset, subset_indices) #remove for final code

        train_size = int(0.8*len(train_dataset))
        val_size = len(train_dataset)-train_size

        train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
        return train_subset, val_subset
    
    def dataloader(self):  
        train_subset, val_subset = self.dataset()
        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

        print (f'loading {self.clinic_id}')
        return train_loader, val_loader

### Models

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [65]:
def resnet18 (weights='DEFAULT'): #James
    
    resnet18 = models.resnet18(weights = weights).to(device)
    for param in resnet18.parameters():
        param.requires_grad = False
    resnet18.fc = nn.Sequential (
    nn.Linear(in_features = 512, out_features = 256, bias = True),
    nn.Dropout(p = 0.5),
    nn.Linear(in_features = 256, out_features = 1, bias = True),
    nn.Sigmoid()
    )
    
    for param in resnet18.fc.parameters():
        param.requires_grad = True
        
    resnet18.__class__.__name__ = 'ResNet18'
    return resnet18

In [66]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, drop_out=0.2):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.drop_out = nn.Dropout(drop_out)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.drop_out(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            out += self.downsample(x)
        out = self.relu(out)
        out = self.drop_out(out)
        return out

class CustomResNet18(nn.Module):
    def __init__(self, drop_out=0.2):
        super(CustomResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Replace residual blocks with custom BasicBlock including dropout
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1, drop_out=drop_out)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2, drop_out=drop_out)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2, drop_out=drop_out)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2, drop_out=drop_out)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 1000)
        

    def _make_layer(self, block, out_channels, num_blocks, stride, drop_out):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, drop_out))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def custom_resnet18(weights='DEFAULT', drop_out=0.2):
    custom_resnet18 = CustomResNet18(drop_out=0.2)
    custom_resnet18.load_state_dict(models.resnet18(weights = weights).state_dict(), strict=False)

# Then freeze parameters
    for param in custom_resnet18.parameters():
        param.requires_grad = False
    
    custom_resnet18.fc = nn.Sequential (
    nn.Linear(in_features = 512, out_features =1),
    nn.Sigmoid()
    )
    for param in custom_resnet18.fc.parameters():
        param.requires_grad = True
    return custom_resnet18



In [67]:
def vgg16(weights = 'DEFAULT'): #David
    vgg16 = models.vgg16(weights=weights).to(device)

# Freeze the parameters of the base model
    for param in vgg16.features.parameters():
        param.requires_grad = False
    
# Modify the classifier part for binary classification
    vgg16.classifier[6] = nn.Sequential(
        nn.Linear(vgg16.classifier[6].in_features, 512),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(512, 1),
        nn.Sigmoid()
    )
    
    vgg16.__class__.__name__ = 'VGG16'
    return vgg16

    
def vgg19 (weights='DEFAULT'):
    vgg19 = models.vgg19 (weights=weights).to(device)
    
    for param in vgg19.parameters():
        param.requires_grad = False
        
    vgg19.classifier = nn.Sequential (
        nn.Linear(25088, 4096),        
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),       
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 1),
        nn.Sigmoid()          
    )
    for param in vgg19.classifier.parameters():
        param.requires_grad = True
    
    vgg19.__class__.__name__ = 'VGG19'
    return vgg19

### Training

In [68]:
import torch
import torch.optim as optim


#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def calculate_accuracy(outputs, labels, threshold=0.5):
    preds = (outputs > threshold).float()
    correct = (preds == labels).float().sum()
    accuracy = correct / labels.size(0)
    return accuracy

def train_local_model(model, train_loader, val_loader, num_epochs=10):
    criterion = torch.nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        total_train = 0
        threshold = 0.5

        for images, labels in train_loader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze(1)
            # Calculate loss
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_accuracy += calculate_accuracy(outputs, labels, threshold)
            total_train += 1


        # Validation phase
        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels.float())
                val_loss += loss.item()
                val_accuracy += calculate_accuracy(outputs, labels, threshold)
                total_val += 1

        avg_val_loss = val_loss / total_val
        avg_val_accuracy = val_accuracy / total_val
        print (f"Epoch {epoch + 1}/{num_epochs}:\ntrain_loss: {running_loss / total_train}, train_accuracy: {running_accuracy / total_train}\nValidation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}")
    
    return model, avg_val_loss, avg_val_accuracy


### Federated Learning

In [69]:
def federated_averaging (client_weights):
    avg_weights = client_weights[0].copy()
    
    for key in avg_weights.keys():
        for key in avg_weights.keys():
            for i in range (1, len (client_weights)):
                avg_weights[key] += client_weights[i][key]
                
            avg_weights[key] = avg_weights[key] / len (client_weights)
            
    return avg_weights

In [70]:
def federated_learning (model, num_clients, num_rounds, train_loaders, val_loaders):
    global_model = model('DEFAULT')
    global_weights = global_model.state_dict()
    
    for round_num in range (num_rounds):
        print (f'Round {round_num+1}')
        
        client_weights = []
        
        for client_id in range (num_clients):
            print (f'client {client_id+1} training...')
            
            local_model = model(None)
            local_model.load_state_dict (global_weights)
            local_model.to(device)
            
            client_train_loader = train_loaders[client_id]
            client_val_loader = val_loaders[client_id]
            
            output_model, _, _ = train_local_model (local_model, client_train_loader, client_val_loader)
            client_updated_weights = output_model.state_dict()
            
            client_weights.append (client_updated_weights)
            
        global_weights = federated_averaging (client_weights)
        
        global_model.load_state_dict (global_weights)
    return global_model

### Experimental Setup

### Evaluation

In [71]:
#Setting up dataset for training the FL model
num_clients = 4

num_rounds = 3

train_loader_0, val_loader_0 = LocalData('clinic_0', range(8050, 8250)).dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loader_1, val_loader_1 = LocalData('clinic_1', range(8050, 8250)).dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loader_2, val_loader_2 = LocalData('clinic_2', range(8050, 8250)).dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loader_3, val_loader_3 = LocalData('clinic_3', range(8050, 8250)).dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loaders = [train_loader_0, train_loader_1, train_loader_2, train_loader_3]
val_loaders = [val_loader_0, val_loader_1, val_loader_2, val_loader_3]

loading clinic_0
loading clinic_1
loading clinic_2
loading clinic_3


In [72]:
# Metrics
def metrics (ground_truths, predictions):
    accuracy = accuracy_score(ground_truths, predictions).round(4)
    precision = precision_score (ground_truths, predictions).round(4)
    recall = recall_score (ground_truths, predictions).round(4)
    f1 = f1_score (ground_truths, predictions).round(4)
    confusion_ma = confusion_matrix (ground_truths, predictions)
    
    print ('Accuracy score: ', accuracy)
    print ('Precision score: ', precision)
    print ('Recall score: ', recall)
    print ('F1 score: ', f1)
    print ('Confusion Matrix: \n', confusion_ma)
    return accuracy, precision, recall, f1
    

In [73]:
#Train client model on clinic4's train_loader
#Make prediction on clinic4's val_loader
def evaluation(client_model):
    client_model.eval()
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for images, labels in val_loader_clinic5:
            images, labels = images.to(device), labels.to(device)
            output = client_model(images)
            output = output.round()
            predictions.append (output)
            ground_truths.append (labels)
    predictions = np.concatenate (predictions).reshape(-1).astype ('int')
    ground_truths = np.concatenate (ground_truths)
    
    return metrics (ground_truths, predictions)

In [74]:
results = []
client_models = [custom_resnet18, resnet18, vgg16, vgg19]

#### 1. Evaluation on the held-out clinic
After several rounds of training, the global model's weights are now used as initiallized weights for a fresh client model. Then, we will use this model to make prediction on the eval dataset on the 5th clinic's data.

In [75]:
#Setting up dataset for evaluation on clinic 5
train_loader_clinic5, val_loader_clinic5 = LocalData ('clinic_4', range (7550, 7750)).dataloader() #remove/change range()

loading clinic_4


In [76]:
for client_model in client_models:
    model_name = client_model().__class__.__name__
    client_model = train_local_model (client_model('DEFAULT'), train_loader_clinic5, val_loader_clinic5)[0]
    accuracy, precision, recall, f1 = evaluation (client_model)
    results.append ([model_name, accuracy, precision, recall, f1])

Epoch 1/10:
train_loss: 0.6908105373382568, train_accuracy: 0.518750011920929
Validation Loss: 0.7273, Validation Accuracy: 0.4375
Epoch 2/10:
train_loss: 0.6929908871650696, train_accuracy: 0.543749988079071
Validation Loss: 0.7303, Validation Accuracy: 0.3906
Epoch 3/10:
train_loss: 0.6838723182678222, train_accuracy: 0.543749988079071
Validation Loss: 0.7595, Validation Accuracy: 0.3594
Epoch 4/10:
train_loss: 0.6876168131828309, train_accuracy: 0.550000011920929
Validation Loss: 0.7332, Validation Accuracy: 0.5312
Epoch 5/10:
train_loss: 0.6905143141746521, train_accuracy: 0.550000011920929
Validation Loss: 0.7088, Validation Accuracy: 0.4531
Epoch 6/10:
train_loss: 0.6859252572059631, train_accuracy: 0.550000011920929
Validation Loss: 0.6971, Validation Accuracy: 0.5312
Epoch 7/10:
train_loss: 0.6959640860557557, train_accuracy: 0.5375000238418579
Validation Loss: 0.7079, Validation Accuracy: 0.4844
Epoch 8/10:
train_loss: 0.6879673004150391, train_accuracy: 0.5625
Validation Loss

In [77]:
for client_model in client_models:
    global_model = federated_learning (client_model, num_clients, 2, train_loaders, val_loaders) #change num_rounds according to your GPU, which mine doesn't have one :<.
    model = train_local_model (global_model, train_loader_clinic5, val_loader_clinic5)[0]
    accuracy, precision, recall, f1 = evaluation (model)
    model_name = f'FedAVG {client_model().__class__.__name__}'
    results.append ([model_name, accuracy, precision, recall, f1])

Round 1
client 1 training...
Epoch 1/10:
train_loss: 0.695528531074524, train_accuracy: 0.46875
Validation Loss: 0.7164, Validation Accuracy: 0.3594
Epoch 2/10:
train_loss: 0.6447457194328308, train_accuracy: 0.7250000238418579
Validation Loss: 0.6339, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.6225085616111755, train_accuracy: 0.731249988079071
Validation Loss: 0.5929, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.603466522693634, train_accuracy: 0.737500011920929
Validation Loss: 0.5971, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5923510909080505, train_accuracy: 0.737500011920929
Validation Loss: 0.5857, Validation Accuracy: 0.7344
Epoch 6/10:
train_loss: 0.5868699669837951, train_accuracy: 0.737500011920929
Validation Loss: 0.5911, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.5822323679924011, train_accuracy: 0.737500011920929
Validation Loss: 0.5997, Validation Accuracy: 0.7344
Epoch 8/10:
train_loss: 0.587666392326355, train_accuracy: 0.7375

Epoch 3/10:
train_loss: 0.5596887469291687, train_accuracy: 0.7562500238418579
Validation Loss: 0.5732, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.5618409872055053, train_accuracy: 0.7562500238418579
Validation Loss: 0.5735, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5547371327877044, train_accuracy: 0.7562500238418579
Validation Loss: 0.5742, Validation Accuracy: 0.7344
Epoch 6/10:
train_loss: 0.564717274904251, train_accuracy: 0.7562500238418579
Validation Loss: 0.5740, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.5587265908718109, train_accuracy: 0.7562500238418579
Validation Loss: 0.5728, Validation Accuracy: 0.7344
Epoch 8/10:
train_loss: 0.5593784093856812, train_accuracy: 0.7562500238418579
Validation Loss: 0.5730, Validation Accuracy: 0.7344
Epoch 9/10:
train_loss: 0.5575137078762055, train_accuracy: 0.7562500238418579
Validation Loss: 0.5730, Validation Accuracy: 0.7344
Epoch 10/10:
train_loss: 0.5537863910198212, train_accuracy: 0.75625002384185

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score:  0.525
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[21  0]
 [19  0]]
Round 1
client 1 training...
Epoch 1/10:
train_loss: 0.6149606585502625, train_accuracy: 0.6937500238418579
Validation Loss: 0.5811, Validation Accuracy: 0.7344
Epoch 2/10:
train_loss: 0.5646928608417511, train_accuracy: 0.737500011920929
Validation Loss: 0.5769, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.5765359818935394, train_accuracy: 0.737500011920929
Validation Loss: 0.5400, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.5148366391658783, train_accuracy: 0.7437499761581421
Validation Loss: 0.5188, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5286639213562012, train_accuracy: 0.7437499761581421
Validation Loss: 0.5038, Validation Accuracy: 0.7500
Epoch 6/10:
train_loss: 0.49799426198005675, train_accuracy: 0.78125
Validation Loss: 0.4875, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.48532121777534487, train_accuracy: 0.7562500238

Epoch 2/10:
train_loss: 0.4403857052326202, train_accuracy: 0.7875000238418579
Validation Loss: 0.4396, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.440521901845932, train_accuracy: 0.7875000238418579
Validation Loss: 0.4227, Validation Accuracy: 0.7656
Epoch 4/10:
train_loss: 0.40672417879104616, train_accuracy: 0.793749988079071
Validation Loss: 0.4182, Validation Accuracy: 0.7656
Epoch 5/10:
train_loss: 0.4239732563495636, train_accuracy: 0.7875000238418579
Validation Loss: 0.4127, Validation Accuracy: 0.8281
Epoch 6/10:
train_loss: 0.36617336273193357, train_accuracy: 0.831250011920929
Validation Loss: 0.4075, Validation Accuracy: 0.8281
Epoch 7/10:
train_loss: 0.3831250250339508, train_accuracy: 0.8125
Validation Loss: 0.4022, Validation Accuracy: 0.8281
Epoch 8/10:
train_loss: 0.37183985114097595, train_accuracy: 0.856249988079071
Validation Loss: 0.3996, Validation Accuracy: 0.8281
Epoch 9/10:
train_loss: 0.35497745871543884, train_accuracy: 0.8500000238418579
Validatio

Epoch 5/10:
train_loss: 0.1244825541973114, train_accuracy: 0.9624999761581421
Validation Loss: 0.4746, Validation Accuracy: 0.8750
Epoch 6/10:
train_loss: 0.04020917136222124, train_accuracy: 1.0
Validation Loss: 0.5475, Validation Accuracy: 0.8750
Epoch 7/10:
train_loss: 0.007853778917342425, train_accuracy: 1.0
Validation Loss: 0.6400, Validation Accuracy: 0.7500
Epoch 8/10:
train_loss: 0.0017754130414687098, train_accuracy: 1.0
Validation Loss: 0.7232, Validation Accuracy: 0.7500
Epoch 9/10:
train_loss: 0.0003988543845480308, train_accuracy: 1.0
Validation Loss: 0.7958, Validation Accuracy: 0.7500
Epoch 10/10:
train_loss: 0.00012990410759812222, train_accuracy: 1.0
Validation Loss: 0.8599, Validation Accuracy: 0.8906
Round 2
client 1 training...
Epoch 1/10:
train_loss: 1.0665751099586487, train_accuracy: 0.71875
Validation Loss: 0.5478, Validation Accuracy: 0.7969
Epoch 2/10:
train_loss: 0.324489039182663, train_accuracy: 0.875
Validation Loss: 0.4968, Validation Accuracy: 0.8438
E

Epoch 10/10:
train_loss: 0.0007718519453192129, train_accuracy: 1.0
Validation Loss: 1.2118, Validation Accuracy: 0.7031
client 2 training...
Epoch 1/10:
train_loss: 0.7209982752799988, train_accuracy: 0.6312500238418579
Validation Loss: 0.4085, Validation Accuracy: 0.8594
Epoch 2/10:
train_loss: 0.48207991719245913, train_accuracy: 0.75
Validation Loss: 0.3756, Validation Accuracy: 0.8594
Epoch 3/10:
train_loss: 0.3794451951980591, train_accuracy: 0.768750011920929
Validation Loss: 0.3194, Validation Accuracy: 0.8906
Epoch 4/10:
train_loss: 0.2219981461763382, train_accuracy: 0.956250011920929
Validation Loss: 0.3168, Validation Accuracy: 0.8281
Epoch 5/10:
train_loss: 0.09979931116104127, train_accuracy: 0.9937499761581421
Validation Loss: 0.3585, Validation Accuracy: 0.8281
Epoch 6/10:
train_loss: 0.032679795660078524, train_accuracy: 0.9937499761581421
Validation Loss: 0.3751, Validation Accuracy: 0.8281
Epoch 7/10:
train_loss: 0.010592721868306398, train_accuracy: 1.0
Validation L

Epoch 5/10:
train_loss: 0.006152716092765331, train_accuracy: 1.0
Validation Loss: 0.2642, Validation Accuracy: 0.8906
Epoch 6/10:
train_loss: 0.0011737538501620294, train_accuracy: 1.0
Validation Loss: 0.2422, Validation Accuracy: 0.8906
Epoch 7/10:
train_loss: 0.0012312305974774064, train_accuracy: 1.0
Validation Loss: 0.2444, Validation Accuracy: 0.9062
Epoch 8/10:
train_loss: 0.0004405231564305723, train_accuracy: 1.0
Validation Loss: 0.3108, Validation Accuracy: 0.9062
Epoch 9/10:
train_loss: 0.00013310056892805732, train_accuracy: 1.0
Validation Loss: 0.4147, Validation Accuracy: 0.9062
Epoch 10/10:
train_loss: 0.00016980124055407942, train_accuracy: 1.0
Validation Loss: 0.4453, Validation Accuracy: 0.9062
Epoch 1/10:
train_loss: 0.7258954107761383, train_accuracy: 0.824999988079071
Validation Loss: 0.2406, Validation Accuracy: 0.9531
Epoch 2/10:
train_loss: 0.16944245249032974, train_accuracy: 0.9375
Validation Loss: 0.3188, Validation Accuracy: 0.8438
Epoch 3/10:
train_loss: 0.

In [78]:
df_eval = pd.DataFrame (columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1'],
                       data = results)

In [79]:
df_eval

,Model,Accuracy,Precision,Recall,F1
0,CustomResNet18,0.400,0.4074,0.5789,0.4783
1,ResNet18,0.850,0.8421,0.8421,0.8421
2,VGG16,0.800,0.7895,0.7895,0.7895
3,VGG19,0.925,0.9444,0.8947,0.9189
4,FedAVG CustomResNet18,0.525,0.0000,0.0000,0.0000
5,FedAVG ResNet18,0.850,0.8824,0.7895,0.8333
6,FedAVG VGG16,0.925,0.9444,0.8947,0.9189
7,FedAVG VGG19,0.925,0.9444,0.8947,0.9189


#### 2. Evaluation on each client's validation data

In [80]:
results_2 = []

In [81]:

for client_model in client_models:
    model_name = client_model().__class__.__name__
    for i in range (4):
        local_model = train_local_model (client_model('DEFAULT'), train_loaders[i], val_loaders[i])[0]
        accuracy, precision, recall, f1 = evaluation (local_model)
        results_2.append ([model_name, f'Clinic_{i}', accuracy, precision, recall, f1])

Epoch 1/10:
train_loss: 0.6383876919746398, train_accuracy: 0.699999988079071
Validation Loss: 0.7351, Validation Accuracy: 0.2969
Epoch 2/10:
train_loss: 0.6029233336448669, train_accuracy: 0.737500011920929
Validation Loss: 0.6139, Validation Accuracy: 0.7188
Epoch 3/10:
train_loss: 0.5831897258758545, train_accuracy: 0.737500011920929
Validation Loss: 0.6077, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.5831525921821594, train_accuracy: 0.737500011920929
Validation Loss: 0.6162, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5863602042198182, train_accuracy: 0.737500011920929
Validation Loss: 0.6212, Validation Accuracy: 0.7344
Epoch 6/10:
train_loss: 0.5856330633163452, train_accuracy: 0.737500011920929
Validation Loss: 0.6376, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.5817507863044739, train_accuracy: 0.737500011920929
Validation Loss: 0.6420, Validation Accuracy: 0.7344
Epoch 8/10:
train_loss: 0.5627174377441406, train_accuracy: 0.737500011920929
Valid

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score:  0.525
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[21  0]
 [19  0]]
Epoch 1/10:
train_loss: 0.5775986433029174, train_accuracy: 0.7437499761581421
Validation Loss: 0.6007, Validation Accuracy: 0.8594
Epoch 2/10:
train_loss: 0.5772770941257477, train_accuracy: 0.7437499761581421
Validation Loss: 0.4579, Validation Accuracy: 0.8594
Epoch 3/10:
train_loss: 0.5605790138244628, train_accuracy: 0.7437499761581421
Validation Loss: 0.3982, Validation Accuracy: 0.8594
Epoch 4/10:
train_loss: 0.5697268903255462, train_accuracy: 0.7437499761581421
Validation Loss: 0.4163, Validation Accuracy: 0.8594
Epoch 5/10:
train_loss: 0.5704399228096009, train_accuracy: 0.7437499761581421
Validation Loss: 0.4332, Validation Accuracy: 0.8594
Epoch 6/10:
train_loss: 0.5602511286735534, train_accuracy: 0.7437499761581421
Validation Loss: 0.4587, Validation Accuracy: 0.8594
Epoch 7/10:
train_loss: 0.571897315979004, train_accuracy: 0.7437499761581421
Validation L

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score:  0.525
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[21  0]
 [19  0]]
Epoch 1/10:
train_loss: 0.6839525103569031, train_accuracy: 0.574999988079071
Validation Loss: 0.8437, Validation Accuracy: 0.1719
Epoch 2/10:
train_loss: 0.6377359509468079, train_accuracy: 0.75
Validation Loss: 0.7535, Validation Accuracy: 0.2188
Epoch 3/10:
train_loss: 0.6045217394828797, train_accuracy: 0.7562500238418579
Validation Loss: 0.7143, Validation Accuracy: 0.3594
Epoch 4/10:
train_loss: 0.5836956143379212, train_accuracy: 0.7562500238418579
Validation Loss: 0.6240, Validation Accuracy: 0.8125
Epoch 5/10:
train_loss: 0.5648852944374084, train_accuracy: 0.7562500238418579
Validation Loss: 0.5584, Validation Accuracy: 0.8281
Epoch 6/10:
train_loss: 0.5656134486198425, train_accuracy: 0.7562500238418579
Validation Loss: 0.5288, Validation Accuracy: 0.8125
Epoch 7/10:
train_loss: 0.5624136447906494, train_accuracy: 0.7562500238418579
Validation Loss: 0.5168, V

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score:  0.525
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[21  0]
 [19  0]]
Epoch 1/10:
train_loss: 0.6208678841590881, train_accuracy: 0.7437499761581421
Validation Loss: 0.5869, Validation Accuracy: 0.7344
Epoch 2/10:
train_loss: 0.5588603913784027, train_accuracy: 0.7562500238418579
Validation Loss: 0.5686, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.557298082113266, train_accuracy: 0.7562500238418579
Validation Loss: 0.5458, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.5175880491733551, train_accuracy: 0.7562500238418579
Validation Loss: 0.5291, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5104659557342529, train_accuracy: 0.78125
Validation Loss: 0.5197, Validation Accuracy: 0.7344
Epoch 6/10:
train_loss: 0.49167664647102355, train_accuracy: 0.7749999761581421
Validation Loss: 0.5052, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.4927061557769775, train_accuracy: 0.762499988079071
Validation Loss: 0.4984

Epoch 10/10:
train_loss: 0.0002840382599970326, train_accuracy: 1.0
Validation Loss: 0.8784, Validation Accuracy: 0.8750
Accuracy score:  0.675
Precision score:  0.875
Recall score:  0.3684
F1 score:  0.5185
Confusion Matrix: 
 [[20  1]
 [12  7]]
Epoch 1/10:
train_loss: 0.6561536192893982, train_accuracy: 0.675000011920929
Validation Loss: 0.5547, Validation Accuracy: 0.7344
Epoch 2/10:
train_loss: 0.44267287850379944, train_accuracy: 0.75
Validation Loss: 0.5038, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.27963329553604127, train_accuracy: 0.90625
Validation Loss: 0.5052, Validation Accuracy: 0.7031
Epoch 4/10:
train_loss: 0.11817112267017364, train_accuracy: 0.987500011920929
Validation Loss: 0.5419, Validation Accuracy: 0.7656
Epoch 5/10:
train_loss: 0.0374671746045351, train_accuracy: 1.0
Validation Loss: 0.7234, Validation Accuracy: 0.7031
Epoch 6/10:
train_loss: 0.010120603162795306, train_accuracy: 1.0
Validation Loss: 0.8813, Validation Accuracy: 0.7031
Epoch 7/10:
t

In [82]:
for client_model in client_models:
    global_model = federated_learning (client_model, num_clients, 2, train_loaders, val_loaders) #change num_rounds according to your GPU, which mine doesn't have one :<.
    for i in range (4):
        model = train_local_model (global_model, train_loaders[i], val_loaders[i])[0]
        accuracy, precision, recall, f1 = evaluation (model)
        model_name = f'FedAVG {client_model().__class__.__name__}'
        results_2.append ([model_name, f'Clinic_{i}', accuracy, precision, recall, f1])

Round 1
client 1 training...
Epoch 1/10:
train_loss: 0.6220854043960571, train_accuracy: 0.731249988079071
Validation Loss: 0.6797, Validation Accuracy: 0.6250
Epoch 2/10:
train_loss: 0.6157195806503296, train_accuracy: 0.737500011920929
Validation Loss: 0.5955, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.6066402792930603, train_accuracy: 0.737500011920929
Validation Loss: 0.5828, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.5821823298931121, train_accuracy: 0.737500011920929
Validation Loss: 0.5795, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5755803108215332, train_accuracy: 0.737500011920929
Validation Loss: 0.5687, Validation Accuracy: 0.7344
Epoch 6/10:
train_loss: 0.5874148845672608, train_accuracy: 0.737500011920929
Validation Loss: 0.5609, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.5743016421794891, train_accuracy: 0.737500011920929
Validation Loss: 0.5696, Validation Accuracy: 0.7344
Epoch 8/10:
train_loss: 0.5757136464118957, train_accu

Epoch 3/10:
train_loss: 0.5649425387382507, train_accuracy: 0.7562500238418579
Validation Loss: 0.5865, Validation Accuracy: 0.7344
Epoch 4/10:
train_loss: 0.5646753251552582, train_accuracy: 0.7562500238418579
Validation Loss: 0.5862, Validation Accuracy: 0.7344
Epoch 5/10:
train_loss: 0.5621638715267181, train_accuracy: 0.7562500238418579
Validation Loss: 0.5853, Validation Accuracy: 0.7344
Epoch 6/10:
train_loss: 0.5500086903572082, train_accuracy: 0.7562500238418579
Validation Loss: 0.5860, Validation Accuracy: 0.7344
Epoch 7/10:
train_loss: 0.5522838473320008, train_accuracy: 0.7562500238418579
Validation Loss: 0.5866, Validation Accuracy: 0.7344
Epoch 8/10:
train_loss: 0.5531079292297363, train_accuracy: 0.7562500238418579
Validation Loss: 0.5868, Validation Accuracy: 0.7344
Epoch 9/10:
train_loss: 0.5596464395523071, train_accuracy: 0.7562500238418579
Validation Loss: 0.5866, Validation Accuracy: 0.7344
Epoch 10/10:
train_loss: 0.5577535450458526, train_accuracy: 0.7562500238418

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy score:  0.525
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[21  0]
 [19  0]]
Epoch 1/10:
train_loss: 0.5776764154434204, train_accuracy: 0.7437499761581421
Validation Loss: 0.4528, Validation Accuracy: 0.8594
Epoch 2/10:
train_loss: 0.5703462779521942, train_accuracy: 0.7437499761581421
Validation Loss: 0.4514, Validation Accuracy: 0.8594
Epoch 3/10:
train_loss: 0.5706385731697082, train_accuracy: 0.7437499761581421
Validation Loss: 0.4541, Validation Accuracy: 0.8594
Epoch 4/10:
train_loss: 0.5736704289913177, train_accuracy: 0.7437499761581421
Validation Loss: 0.4544, Validation Accuracy: 0.8594
Epoch 5/10:
train_loss: 0.5684538960456849, train_accuracy: 0.7437499761581421
Validation Loss: 0.4539, Validation Accuracy: 0.8594
Epoch 6/10:
train_loss: 0.5758851528167724, train_accuracy: 0.7437499761581421
Validation Loss: 0.4548, Validation Accuracy: 0.8594
Epoch 7/10:
train_loss: 0.5663521945476532, train_accuracy: 0.7437499761581421
Validation 

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 2/10:
train_loss: 0.4143748998641968, train_accuracy: 0.793749988079071
Validation Loss: 0.4540, Validation Accuracy: 0.7500
Epoch 3/10:
train_loss: 0.38778656125068667, train_accuracy: 0.824999988079071
Validation Loss: 0.4452, Validation Accuracy: 0.7500
Epoch 4/10:
train_loss: 0.39513585567474363, train_accuracy: 0.800000011920929
Validation Loss: 0.4364, Validation Accuracy: 0.7500
Epoch 5/10:
train_loss: 0.3660979688167572, train_accuracy: 0.831250011920929
Validation Loss: 0.4348, Validation Accuracy: 0.7500
Epoch 6/10:
train_loss: 0.3593867003917694, train_accuracy: 0.84375
Validation Loss: 0.4278, Validation Accuracy: 0.7500
Epoch 7/10:
train_loss: 0.356302547454834, train_accuracy: 0.8500000238418579
Validation Loss: 0.4182, Validation Accuracy: 0.7500
Epoch 8/10:
train_loss: 0.35902122855186464, train_accuracy: 0.8500000238418579
Validation Loss: 0.4132, Validation Accuracy: 0.7500
Epoch 9/10:
train_loss: 0.35227404832839965, train_accuracy: 0.84375
Validation Loss: 0.4

Epoch 1/10:
train_loss: 0.616792368888855, train_accuracy: 0.668749988079071
Validation Loss: 0.5829, Validation Accuracy: 0.7344
Epoch 2/10:
train_loss: 0.5004443764686585, train_accuracy: 0.737500011920929
Validation Loss: 0.5139, Validation Accuracy: 0.7344
Epoch 3/10:
train_loss: 0.3816349685192108, train_accuracy: 0.762499988079071
Validation Loss: 0.4713, Validation Accuracy: 0.7500
Epoch 4/10:
train_loss: 0.2832833230495453, train_accuracy: 0.84375
Validation Loss: 0.5484, Validation Accuracy: 0.8125
Epoch 5/10:
train_loss: 0.16284437477588654, train_accuracy: 0.9375
Validation Loss: 0.5609, Validation Accuracy: 0.7656
Epoch 6/10:
train_loss: 0.08486959487199783, train_accuracy: 0.987500011920929
Validation Loss: 0.7144, Validation Accuracy: 0.8750
Epoch 7/10:
train_loss: 0.025048345513641834, train_accuracy: 1.0
Validation Loss: 0.8575, Validation Accuracy: 0.8750
Epoch 8/10:
train_loss: 0.0033974813530221583, train_accuracy: 1.0
Validation Loss: 0.9190, Validation Accuracy: 0.

Epoch 6/10:
train_loss: 0.0002974273811560124, train_accuracy: 1.0
Validation Loss: 1.0927, Validation Accuracy: 0.6406
Epoch 7/10:
train_loss: 0.00021514506952371448, train_accuracy: 1.0
Validation Loss: 1.1641, Validation Accuracy: 0.6406
Epoch 8/10:
train_loss: 0.00018436437676427886, train_accuracy: 1.0
Validation Loss: 1.2246, Validation Accuracy: 0.6406
Epoch 9/10:
train_loss: 0.00013370063461479731, train_accuracy: 1.0
Validation Loss: 1.2765, Validation Accuracy: 0.6406
Epoch 10/10:
train_loss: 7.624515601492021e-05, train_accuracy: 1.0
Validation Loss: 1.3248, Validation Accuracy: 0.6406
client 4 training...
Epoch 1/10:
train_loss: 0.13089406341314316, train_accuracy: 0.9375
Validation Loss: 0.7826, Validation Accuracy: 0.8281
Epoch 2/10:
train_loss: 0.03734641969203949, train_accuracy: 0.9937499761581421
Validation Loss: 0.7210, Validation Accuracy: 0.7500
Epoch 3/10:
train_loss: 0.004591573099605739, train_accuracy: 1.0
Validation Loss: 0.7557, Validation Accuracy: 0.7500
Ep

Epoch 9/10:
train_loss: 0.0005431801953818649, train_accuracy: 1.0
Validation Loss: 1.0516, Validation Accuracy: 0.7031
Epoch 10/10:
train_loss: 0.0003335475077619776, train_accuracy: 1.0
Validation Loss: 1.1346, Validation Accuracy: 0.7031
client 2 training...
Epoch 1/10:
train_loss: 0.6732498645782471, train_accuracy: 0.6499999761581421
Validation Loss: 0.3805, Validation Accuracy: 0.8594
Epoch 2/10:
train_loss: 0.448753809928894, train_accuracy: 0.7562500238418579
Validation Loss: 0.3482, Validation Accuracy: 0.8594
Epoch 3/10:
train_loss: 0.32880387604236605, train_accuracy: 0.8062499761581421
Validation Loss: 0.3334, Validation Accuracy: 0.8125
Epoch 4/10:
train_loss: 0.18845855593681335, train_accuracy: 0.981249988079071
Validation Loss: 0.3668, Validation Accuracy: 0.8906
Epoch 5/10:
train_loss: 0.08765230476856231, train_accuracy: 0.981249988079071
Validation Loss: 0.3523, Validation Accuracy: 0.8125
Epoch 6/10:
train_loss: 0.03468024767935276, train_accuracy: 1.0
Validation Lo

Epoch 4/10:
train_loss: 0.019512821518583225, train_accuracy: 1.0
Validation Loss: 0.4026, Validation Accuracy: 0.9219
Epoch 5/10:
train_loss: 0.0023652797448448838, train_accuracy: 1.0
Validation Loss: 0.2487, Validation Accuracy: 0.9062
Epoch 6/10:
train_loss: 0.002063631173223257, train_accuracy: 1.0
Validation Loss: 0.2995, Validation Accuracy: 0.9219
Epoch 7/10:
train_loss: 0.0009863295359537005, train_accuracy: 1.0
Validation Loss: 0.3000, Validation Accuracy: 0.9062
Epoch 8/10:
train_loss: 0.00016233352944254874, train_accuracy: 1.0
Validation Loss: 0.3859, Validation Accuracy: 0.8906
Epoch 9/10:
train_loss: 9.446868571103551e-05, train_accuracy: 1.0
Validation Loss: 0.4847, Validation Accuracy: 0.9062
Epoch 10/10:
train_loss: 6.788504142605234e-05, train_accuracy: 1.0
Validation Loss: 0.5262, Validation Accuracy: 0.9062
Epoch 1/10:
train_loss: 0.7299207448959351, train_accuracy: 0.8125
Validation Loss: 0.6082, Validation Accuracy: 0.7812
Epoch 2/10:
train_loss: 0.13915458768606

In [83]:
df_eval_2 = pd.DataFrame (columns = ['Model', 'Clinic', 'Accuracy', 'Precision', 'Recall', 'F1'],
                         data = results_2)

In [84]:
resnet18()

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)


In [90]:
a = [1, 2, 3]
b = [4, 5, 6]

for c, d in zip (a, b):
    print (c, d)

1 4
2 5
3 6
